# **Introduction**

The aim of this project is to investigate the development of hybrid models of probabilistic with quantum computations to explore the potential for enhancing the computational power of current state-of-the-art NISQ computing.

These models are primarly used for solving the computationally hard problems, which can be performed as combinatorial optimization problems, where the search space is finite, but too large to observe all states. 

In this publication, we consider the implementation approach on the example of the number partitioning problem. 

# **Method**
![Combinatorial optimization](src/rolos_image.png)
The idea of probabilistic computing is to create a circuit of so-called p-bits. They can be perceived as stochastic building blocks with a normalized output $m_i \in \{-1, 1\}$, which is calculated from the current state of the circuit using input-output relations of p-bits. We will call the state of the system the current ordered set of p-bit values.

The **energy function** is a function of the variables of the system whose global extremum corresponds to the solution of the optimization problem we consider.

Hence, when updating p-bits, we generate new system states with the aim of identifying states that **minimize the energy** as defined by the optimization problem.


It appears that there exists a class of problems where the energy is quadratic with respect to each of the parameters. They are called quadratic energy models or the **Ising
model**, and they are often mapped to NP-hard problems. 

$$E(m_1, m_2, \ldots) = -\sum_{i \leq j} (W_{ij} m_i m_j + h_i m_i)
$$


It is defined by two parameters: weight matrix W and bias vector h.

**Example: number partitioning problem**


Number partitioning is the task of deciding whether a given multiset $S = \{a_1, ...,  a_n\}$ of positive integers can be partitioned into two subsets $S_1$ and $S_2$ such that the sum of the numbers in $S_1$ equals the sum of the numbers in $S_2$.

Hence we can define the energy function as $(\sum_{i} a_i m_i)^2$. Note that $m_i \in \{-1, 1\}$, hence the minimum of energy is zero $\iff$ the partitioning exists.

Now that is easy to see that we can define the matrix $W$: $W_{ij} = - a_i a_j$ and the bias vector $h$: $h_i = 0$. 


# **Algorithm**

- **Determine the energy function, the minimum of which will correspond to the solution of the given problem.**

In the considered example it is $$E(m_1, m_2, \ldots, m_n) = (\sum_{i} a_i m_i)^2$$ The key is to determine weights matrix W and bias vector h.

- **Define the initial state of the p-bits**.

 $\forall i$ $m_i = 1$ is good choice.

- **Choose the order of p-bit state changing** 

The order in which p-bits change their states in a p-bit computer architecture depends on random permutations of indices for each step. This implementation ensures that every p-bit is updated an equal number of times as other bits. This approach promotes fairness in state transitions, prevents bias towards specific p-bits, and balances the computational load across them.

- **For each p-bit, calculate the new output using *activation function***

    $$m_i = f(\beta, I_i) = sign(\tanh(\beta I_i) - rand([-1, 1]))$$
Where $I_i$ is the input of each bit and is a negated partial derivative with respect to p-bit $m_i$. It is called *synaptic input*.
    $$I_i = -\frac{\delta E(m_1, \ldots, m_n)}{\delta m_i}$$
and $\beta$ is a inverse temperature - an important parameter in the algorithm, because it defines the fluctuation behaviour and helps to achieve the energy global minimum. 

- **Increase $\beta$ (cool the system) if the new state has a lower energy**

At high temperatures updates that change the energy of the system are comparatively more probable. When the system is highly correlated, updates are rejected and the simulation is said to suffer from critical slowing down. Hence we are starting with low inverse temperature $\beta$ and then after every successful energy change we increase it's value to help the system to obtain the minimal energy value.

- **Return the state that corresponds to the minimum energy achieved.**

    The ability of this approach to produce correct results is based primarily on Boltzmann's law, which says that at a certain temperature ($1 / \beta$) the states that minimise energy occur with the highest probability.
    
        


# **Results**

After putting the described algorithm into practice, I tested it across various scenarios using the number partitioning theorem. Remarkably, in each case, it managed to reach the minimum energy state, consistently providing accurate solutions. To illustrate, let's consider the example of the output when working with numbers from 1 to 32.

First set of numbers:
**2 3 4 5 6 7 10 13 15 16 21 23 24 27 28 29 31**

Second set of numbers: 
**1 8 9 11 12 14 17 18 19 20 22 25 26 30 32**

Now, let's examine the computational contrast between our solution and the naive implementation (complete search).

In [2]:
import pandas as pd
import altair as alt

test_cases = [6, 8, 12, 18, 19, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32]

execution_times_complete_search = [5.7697296142578125e-05, 7.700920104980469e-05, 0.0006759166717529297, 0.03530168533325195, 0.05919766426086426, 0.2158496379852295, 0.24437642097473145, 0.5796020030975342, 1.0086231231689453, 16.80867075920105, 39.62793040275574, 69.11628222465515, 166.19825863838196, 294.98395586013794, 665.634993314743]
execution_times_find_energy_minimum = [0.18785452842712402, 0.22379279136657715, 0.34358716011047363, 0.5495147705078125, 0.5546219348907471, 0.5766348838806152, 0.7116522789001465, 0.7572066783905029, 0.6926448345184326, 0.9135968685150146, 0.964179277420044, 0.9963042736053467, 1.0487761497497559, 1.011204719543457, 1.041233777999878]

data = pd.DataFrame({
    'Input Cases': test_cases,
    'Complete Search Execution Time': execution_times_complete_search,
    'Probabilistic Computing Execution Time': execution_times_find_energy_minimum
})

chart = alt.Chart(data).mark_line(point=True).encode(
    x='Input Cases',
    y='Complete Search Execution Time:Q',
    color=alt.value('blue'),
    tooltip=['Input Cases', 'Complete Search Execution Time']
).properties(
    title='Computational Difference Between Complete Search and Probabilistic Computing',
    width=500
) + alt.Chart(data).mark_line(point=True).encode(
    x='Input Cases',
    y='Probabilistic Computing Execution Time',
    color=alt.value('red'),
    tooltip=['Input Cases', 'Probabilistic Computing Execution Time']
)

chart.interactive()

alt.LayerChart(...)

# **Conclusion**

The main ideas that are associated with the use of probabilistic networks have been reviewed, as well as approaches that reduce the cost of finding the optimal state.  The possibility of searching for the minimum state of energy expressed by the Ising model allows solving combinatorial optimization problems by determining only the parameters of weights and biases. 

A potential continuation of this review is to consider parameter setting via transformation of problems formulated using invertible logic circuits to Ising Model formulation.  

### **Notebooks**

Notebook [factorisation_beta.ipynb](Implementing-Optimization-Algorithms-Using-Probability-Computing/factorisation_beta.ipynb) presents an example of solving Integer factorisation problem using the energy function, which prevents it from being considered as an ising model, because it is not linear.

Notebook [ising_model_and_gate.ipynb](Implementing-Optimization-Algorithms-Using-Probability-Computing/ising_model_beta.ipynb) presents an example of using Ising Model for the implementing invertible AND-gate behaviour.

Notebook [ising_model_np_problems.ipynb](Implementing-Optimization-Algorithms-Using-Probability-Computing/ising_model_np_problems.ipynb) presents an example of using Ising Model for the implementing Number Partitioning and Minimum Bisection Problem


.